In [1]:
import tensorflow as tf
import numpy as np

In [2]:
model_1 = tf.keras.models.load_model("./Sentiment_Classifier_v1")
model_3 = tf.keras.models.load_model("./Sentiment_Classifier_v2")

2025-07-03 19:07:24.439576: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-07-03 19:07:24.439599: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-07-03 19:07:24.439605: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-07-03 19:07:24.439911: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-07-03 19:07:24.440488: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-07-03 19:07:25.415943: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


### Sentiment Testing

In [3]:
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
import nltk, string

punctuation = string.punctuation
stopwords = nltk.corpus.stopwords.words('english')
lm = WordNetLemmatizer()

def clean_text(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = "".join([char.lower() for char in text if char not in punctuation])
    tokens = word_tokenize(text)
    text = " ".join([lm.lemmatize(char) for char in tokens if char not in stopwords])
    return text

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/animesh/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
clean_text("I was left on the edge of my seat throughout")

'left edge seat throughout'

In [9]:
Test_text = [
    'I was left on the edge of my seat throughout',
    'The cinematographer’s lensing captured visceral emotion',
    'I hated the first half, yet the ending made it worth watching.',
    ##Sarcasm
    'Well, that was a disaster wrapped in fancy cinematography',
]



for query in Test_text:
    
    
    ##Model 1 Prediction
    prediction = model_1.predict(np.array([clean_text(query)]))
    print("\nQuery:", query)
    print("Model 1:")
    if(prediction[0]>0.5):
        print("Positive Sentiment", "\nScore:",prediction[0][0])
    else:
        print("Negative Sentiment","\nScore:",prediction[0][0])
    
    ##Model 2 Prediction
    print("\nModel 2:")
    prediction = model_3.predict(np.array([clean_text(query)]))
    if(prediction[0]>0.5):
        print("Positive Sentiment", "\nScore:",prediction[0][0])
    else:
        print("Negative Sentiment","\nScore:",prediction[0][0])

1/1 [==============================] - 1s 1s/step

Query: I was left on the edge of my seat throughout
Model 1:
Positive Sentiment 
Score: 0.99069947

Model 2:
1/1 [==============================] - 3s 3s/step
Positive Sentiment 
Score: 0.9386788
1/1 [==============================] - 0s 13ms/step

Query: The cinematographer’s lensing captured visceral emotion
Model 1:
Positive Sentiment 
Score: 0.95213175

Model 2:
1/1 [==============================] - 2s 2s/step
Positive Sentiment 
Score: 0.9944478
1/1 [==============================] - 0s 14ms/step

Query: I hated the first half, yet the ending made it worth watching.
Model 1:
Negative Sentiment 
Score: 0.0012903499

Model 2:
1/1 [==============================] - 2s 2s/step
Positive Sentiment 
Score: 0.6605513
1/1 [==============================] - 0s 13ms/step

Query: Well, that was a disaster wrapped in fancy cinematography
Model 1:
Positive Sentiment 
Score: 0.99102986

Model 2:
1/1 [==============================] - 2s 2s/step

#### Embedding initialization

- Part 1’s Embedding(vocab_size, 100) layer is randomly initialized and must learn word vectors from scratch.

- Part 3 trains a Word2Vec (CBOW) model on your cleaned, tokenized corpus (vector size = 200), builds an embedding_matrix from those vectors, and then do
- Benefit: Starting from Word2Vec gives you semantically rich vectors (e.g. “good” is close to “great”), so the LSTM can focus on learning classification patterns rather than also discovering basic word semantics.

#### Embedding dimension
- Part 1 uses 100-dim embeddings.
- Part 3 uses 200-dim vectors from Word2Vec.
- Benefit: Higher-dimensional embeddings can encode more nuanced meaning (at the cost of a few more parameters), which often helps classification.